In [4]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import random
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from tabulate import tabulate 
from sklearn.externals import joblib
import joblib as job
import pickle as pk
import warnings
warnings.filterwarnings("ignore")

class Regression(object):
    """docstring for Regression."""
    
    def __init__(self, path,split,Coff):
      super(Regression, self).__init__()
      self.path = path
      self.split = split


    def NfoldSplit(self,Data,n):
        
      NewData = []
      size = (Data.shape[0]//n)+1
      for i in range(n):
        T = Data[i*size:i*size+size]
        NewData.append(T)
      return NewData

    def MSE(self,ytrue,ypred):

      MSE = np.square(ytrue-ypred)
      MSE = np.sum(MSE)/len(ytrue)
      return MSE

    def Load_DataSet(self,path):
    
      File = open(path, "r")
      L = File.readlines() 

      Data_List = []
      for Lines in L:
        Data_List.append(Lines.strip().split(' '))
      
      l = len(Data_List[0])
      col = [str(i) for i in range(1,l+1) ]
      Data = pd.DataFrame(Data_List,columns=col)  
      #print(Data)

      Data = pd.get_dummies(Data, columns=['1'])  
      Data = Data.astype(float)
      
      return Data

    def fit(self):

      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()           

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        LR = LinearRegression(fit_intercept=False).fit(Train[X], Train[Y])
        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_b/'+str(i)
        job.dump(LR.coef_, path+'.pkl')


    def Normal(self):
      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        TrainX = Train[X]
        TrainY = Train[Y]
        TestX = Test[X]
        TestY = Test[Y]

        TrainX = TrainX.to_numpy()
        TrainY = TrainY.to_numpy()
        
        TrainX_Tr = TrainX.T

        Coff = np.matmul(TrainX_Tr,TrainX)
        Coff = np.linalg.inv(Coff)
        Coff = np.matmul(Coff,TrainX_Tr)
        Coff = np.matmul(Coff,TrainY)

        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_c/'+str(i)
        job.dump(Coff, path+'.pkl')

    def SKlearn(self):
      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        LR = LinearRegression(fit_intercept=False).fit(Train[X], Train[Y])
        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_d/'+str(i)
        job.dump(LR, path+'.pkl')

    def Load(self):
      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()

      # Userdefined MSE
      MSETrainU = []
      MSEValiU = []

      # Predefined MSE
      MSETrainP = []
      MSEValiP = []
    
      MSEDataU = pd.DataFrame();
      MSEDataP = pd.DataFrame();

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_b/'+str(i)
        Coff = job.load(path+'.pkl')
        self.Coff = Coff
        
        YTrainP = self.predict(Train[X])
        YTestP = self.predict(Test[X])
        
        MSETrainU.append(self.MSE(Train[Y],YTrainP))
        MSEValiU.append(self.MSE(Test[Y],YTestP))

        MSETrainP.append(mean_squared_error(Train[Y],YTrainP))
        MSEValiP.append(mean_squared_error(Test[Y],YTestP))
      
      MSEDataU['Fold'] = [i+1 for i in range(self.split)]
      MSEDataU['Training'] = MSETrainU
      MSEDataU['Test'] = MSEValiU

      MSEDataP['Fold'] = [i+1 for i in range(self.split)]
      MSEDataP['Training'] = MSETrainP
      MSEDataP['Test'] = MSEValiP

      print('Table Stats for User Defined MSE Function with SKlearn :')
      print(tabulate(MSEDataU , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataU['Training'].mean())
      print('Mean Testing MSE : ',MSEDataU['Test'].mean())
      print('\n')
      print('Table Stats for SKlearn MSE Function with SKlearn :')
      print(tabulate(MSEDataP , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataP['Training'].mean())
      print('Mean Testing MSE : ',MSEDataP['Test'].mean())
      print('\n')

    def LoadNormal(self):
      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()

      # Userdefined MSE
      MSETrainU = []
      MSEValiU = []

      # Predefined MSE
      MSETrainP = []
      MSEValiP = []
    
      MSEDataU = pd.DataFrame();
      MSEDataP = pd.DataFrame();

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        TrainX = Train[X]
        TrainY = Train[Y]
        TestX = Test[X]
        TestY = Test[Y]

        TrainX = TrainX.to_numpy()
        TrainY = TrainY.to_numpy()
  
        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_c/'+str(i)
        Coff = job.load(path+'.pkl')
        self.Coff = Coff

        YTrainP = self.predict(Train[X])
        YTestP = self.predict(Test[X])
        
        
        MSETrainU.append(self.MSE(Train[Y],YTrainP))
        MSEValiU.append(self.MSE(Test[Y],YTestP))

        MSETrainP.append(mean_squared_error(Train[Y],YTrainP))
        MSEValiP.append(mean_squared_error(Test[Y],YTestP))
      
      MSEDataU['Fold'] = [i+1 for i in range(self.split)]
      MSEDataU['Training'] = MSETrainU
      MSEDataU['Test'] = MSEValiU

      MSEDataP['Fold'] = [i+1 for i in range(self.split)]
      MSEDataP['Training'] = MSETrainP
      MSEDataP['Test'] = MSEValiP

      print('Table Stats for User Defined MSE Function with Normal Form :')
      print(tabulate(MSEDataU , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataU['Training'].mean())
      print('Mean Testing MSE : ',MSEDataU['Test'].mean())
      print('\n')
      print('Table Stats for SKlearn MSE Function with Normal Form :')
      print(tabulate(MSEDataP , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataP['Training'].mean())
      print('Mean Testing MSE : ',MSEDataP['Test'].mean()) 
      print('\n')

    def LoadSKlearn(self):
      Data = self.Load_DataSet(self.path)
      
      NewData = self.NfoldSplit(Data,self.split)
      
      # Generate the Target and Data Columns
      Y = ['9']
      X = []
      for i in NewData[0].columns:
        if i not in Y:
          X.append(i)
      Y = '9'
      
      #print(len(X))
      NewData_C = NewData.copy()

      # Userdefined MSE
      MSETrainU = []
      MSEValiU = []

      # Predefined MSE
      MSETrainP = []
      MSEValiP = []
    
      MSEDataU = pd.DataFrame();
      MSEDataP = pd.DataFrame();

      for i in range(self.split):
        NewData = NewData_C.copy()  
        Test = NewData.pop(i)
        Train = pd.concat(NewData)
        
        path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Saved_Data/1_d/'+str(i)
        LR = job.load(path+'.pkl')
        self.Coff = LR.coef_
        
        YTrainP = LR.predict(Train[X])
        YTestP = LR.predict(Test[X])
        
        MSETrainU.append(self.MSE(Train[Y],YTrainP))
        MSEValiU.append(self.MSE(Test[Y],YTestP))

        MSETrainP.append(mean_squared_error(Train[Y],YTrainP))
        MSEValiP.append(mean_squared_error(Test[Y],YTestP))
      
      MSEDataU['Fold'] = [i+1 for i in range(self.split)]
      MSEDataU['Training'] = MSETrainU
      MSEDataU['Test'] = MSEValiU

      MSEDataP['Fold'] = [i+1 for i in range(self.split)]
      MSEDataP['Training'] = MSETrainP
      MSEDataP['Test'] = MSEValiP

      print('Table Stats for User Defined MSE Function with SKlearn Predict Function :')
      print(tabulate(MSEDataU , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataU['Training'].mean())
      print('Mean Testing MSE : ',MSEDataU['Test'].mean())
      print('\n')
      print('Table Stats for SKlearn MSE Function with SKlearn Predict Function :')
      print(tabulate(MSEDataP , tablefmt = 'psql',headers = 'keys'))
      print('Mean Training MSE : ',MSEDataP['Training'].mean())
      print('Mean Testing MSE : ',MSEDataP['Test'].mean())
      print('\n')


    def predict(self,X_test):
      X_test = X_test.to_numpy()
      y_predicted = (np.sum(X_test*self.Coff,axis = 1))
      return y_predicted

path  = "/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_2/Assignment_2_datasets/regression_data/Dataset.data"
split = 5
R =  Regression(path,split,None)

# 1(b)
#R.fit()

# 1(c)
#R.Normal()

# 1(d)
#R.SKlearn()

R.Load()
#R.LoadNormal()
#R.LoadSKlearn()



Table Stats for User Defined MSE Function with SKlearn :
+----+--------+------------+---------+
|    |   Fold |   Training |    Test |
|----+--------+------------+---------|
|  0 |      1 |    3.76203 | 9.77494 |
|  1 |      2 |    5.30721 | 2.98859 |
|  2 |      3 |    4.61405 | 5.8065  |
|  3 |      4 |    5.06428 | 3.82279 |
|  4 |      5 |    5.02944 | 3.95344 |
+----+--------+------------+---------+
Mean Training MSE :  4.755400001112425
Mean Testing MSE :  5.2692531178154995


Table Stats for SKlearn MSE Function with SKlearn :
+----+--------+------------+---------+
|    |   Fold |   Training |    Test |
|----+--------+------------+---------|
|  0 |      1 |    3.76203 | 9.77494 |
|  1 |      2 |    5.30721 | 2.98859 |
|  2 |      3 |    4.61405 | 5.8065  |
|  3 |      4 |    5.06428 | 3.82279 |
|  4 |      5 |    5.02944 | 3.95344 |
+----+--------+------------+---------+
Mean Training MSE :  4.755400001112425
Mean Testing MSE :  5.2692531178154995


